<a href="https://colab.research.google.com/github/vserta/webscraping/blob/main/WebScrapingZapImoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
url = "https://www.zapimoveis.com.br/aluguel/apartamentos/rj+niteroi/?onde=,Rio%20de%20Janeiro,Niter%C3%B3i,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ENiteroi,-22.885927,-43.115259&transacao=Aluguel&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Apartamento&"
pageTree = requests.get(url, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

In [41]:
def converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,area_lista):
    data = {'aluguel':aluguel_lista,
           'quartos':quartos_lista,
           'endereco':endereco_lista,
           'banheiro':banheiro_lista,
           'area':area_lista}
    df = pd.DataFrame(data)
    return df

In [51]:
def scrap(url):
    pageTree = requests.get(url, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    
    aluguel_tag = pageSoup.find_all('p',{'class':'simple-card__price js-price heading-regular heading-regular__bolder align-left'}) 
    aluguel_lista = [p.text for p in aluguel_tag] 
    #print(len(aluguel_lista)) 
    
    quartos_tag = pageSoup.find_all('li',{'class':'feature__item text-small js-bedrooms'}) 
    quartos_lista = [en.text for en in quartos_tag] 
    #print(len(quartos_lista)) 
    
    endereco_tag = pageSoup.find_all('p',{'class':'color-dark text-regular simple-card__address'}) 
    endereco_lista = [fg.text for fg in endereco_tag] 
    #print(len(endereco_lista)) 
    
    banheiro_tag = pageSoup.find_all('li',{'class':'feature__item text-small js-bathrooms'}) 
    banheiro_lista = [yp.text for yp in banheiro_tag] 
    #print(len(banheiro_lista))   

    area_tag = pageSoup.find_all('li',{'class':'feature__item text-small js-areas'}) 
    area_lista = [yp.text for yp in area_tag] 


    ### Vamos ter criar uma função para transformar listas em df
    df=converter_df(aluguel_lista,quartos_lista,endereco_lista,banheiro_lista,area_lista)
    
    return df

In [43]:
URL = "https://www.zapimoveis.com.br/aluguel/apartamentos/rj+niteroi/?onde=,Rio%20de%20Janeiro,Niter%C3%B3i,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ENiteroi,-22.885927,-43.115259&transacao=Aluguel&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Apartamento&{}"

In [24]:
paginas = ['']
for i in range(2,11):
    a = '?pagina='+str(i)
    print(a)
    paginas.append(a)

?pagina=2
?pagina=3
?pagina=4
?pagina=5
?pagina=6
?pagina=7
?pagina=8
?pagina=9
?pagina=10


In [44]:
df_lista = []
for pg in paginas:
    url = URL.format(pg)
    print('Coletando URL:', url)
    try:
        df = scrap(url)
    except Exception as e:
        print('Erro ao coletar dados na url:', url)
        print(e)
    df_lista.append(df)

df_final = pd.concat(df_lista)

Coletando URL: https://www.zapimoveis.com.br/aluguel/apartamentos/rj+niteroi/?onde=,Rio%20de%20Janeiro,Niter%C3%B3i,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ENiteroi,-22.885927,-43.115259&transacao=Aluguel&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Apartamento&
Coletando URL: https://www.zapimoveis.com.br/aluguel/apartamentos/rj+niteroi/?onde=,Rio%20de%20Janeiro,Niter%C3%B3i,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ENiteroi,-22.885927,-43.115259&transacao=Aluguel&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Apartamento&?pagina=2
Coletando URL: https://www.zapimoveis.com.br/aluguel/apartamentos/rj+niteroi/?onde=,Rio%20de%20Janeiro,Niter%C3%B3i,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ENiteroi,-22.885927,-43.115259&transacao=Aluguel&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Apartamento&?pagina=3
Coletando URL: https://www.zapimoveis.com.br/aluguel/apartamentos/rj+niteroi/?onde=,Rio%20de%20Janeiro,Niter%C3%B3i,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ENiteroi,-22.885927,-43.115259&transa

In [49]:
df_final = df_final.replace('\n','',regex=True)

In [50]:
df_final.head()

,aluguel,quartos,endereco,banheiro,area
0,R$ 1.700 /mês,2,"Avenida Jornalista Alberto Francisco Torres, I...",2,82 m²
1,R$ 1.500 /mês,2,"Travessa Dom Bosco, Icaraí",3,90 m²
2,R$ 1.150 /mês,2,"Rua Tiradentes, Ingá",2,70 m²
3,R$ 3.000 /mês,2,"Santa Rosa, Niterói",2,98 m²
4,R$ 1.500 /mês,2,"Rua Roberto Rowley Mendes, Boa Viagem",2,70 m²
